In [203]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Bidirectional, LSTM, Reshape, RepeatVector, TimeDistributed
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
tf.compat.v1.enable_eager_execution() 

import matplotlib.pyplot as plt

import sys

import numpy as np
import pandas as pd

import os

from PIL import Image

# Load Data
Loading from preprocessed numpy array

In [204]:
def load_data():
    x_train = np.array(pd.read_csv("./data_stock/SP500_average.csv",).drop([0]).drop(columns=['Date']))
    return x_train
            

def split_time_series(t, arr) -> (np.array, np.array):
    a = []
    b = []
    for i in range(len(arr)-t):
        a.append(arr[i: i+t])
        b.append(arr[i+t])
    return (np.array(a), np.array(b))
    

def merge_time_series(arr1, arr2) -> tf.float64 :
    t1 = tf.cast(arr1, 'float64') if tf.is_tensor(arr1) else tf.convert_to_tensor(arr1, dtype='float64')
    t2 = tf.cast(arr2, 'float64') if tf.is_tensor(arr2) else tf.convert_to_tensor(arr2, dtype='float64')
    return tf.concat([t1, tf.expand_dims(t2, 1) ], axis=1)
                                
split = split_time_series(5, load_data())
merged = merge_time_series(*split)

print(split)
print(merged)

(array([[[4366.64, 4411.01, 4287.11, 4356.45],
        [4356.32, 4417.35, 4222.62, 4410.13],
        [4471.38, 4494.52, 4395.34, 4397.94],
        [4547.35, 4602.11, 4477.95, 4482.73],
        [4588.03, 4611.55, 4530.2 , 4532.76]],

       [[4356.32, 4417.35, 4222.62, 4410.13],
        [4471.38, 4494.52, 4395.34, 4397.94],
        [4547.35, 4602.11, 4477.95, 4482.73],
        [4588.03, 4611.55, 4530.2 , 4532.76],
        [4632.24, 4632.24, 4568.7 , 4577.11]],

       [[4471.38, 4494.52, 4395.34, 4397.94],
        [4547.35, 4602.11, 4477.95, 4482.73],
        [4588.03, 4611.55, 4530.2 , 4532.76],
        [4632.24, 4632.24, 4568.7 , 4577.11],
        [4637.99, 4665.13, 4614.75, 4662.85]],

       ...,

       [[2457.77, 2571.42, 2407.53, 2475.56],
        [2344.44, 2449.71, 2344.44, 2447.33],
        [2290.71, 2300.73, 2191.86, 2237.4 ],
        [2431.94, 2453.01, 2295.56, 2304.92],
        [2393.48, 2466.97, 2319.78, 2409.39]],

       [[2344.44, 2449.71, 2344.44, 2447.33],
        [229

# Creating GAN

In [ ]:
class LSTMGAN():
    def __init__(self, t, f, data):
        # standardize data
        self.scaler = StandardScaler()
        self.scaler.fit(data)
        self.data = self.scaler.transform(data)
        
        self.time_series_len = t
        self.feature_len = f
        self.gen_shape = (self.time_series_len, self.feature_len)
        self.dis_shape = (self.time_series_len+1, self.feature_len)

        optimizer = Adam(0.0001, 0.4)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates song
        real_input = Input(shape=self.gen_shape)
        gen_output = self.generator(real_input)
        print(gen_output)
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(merge_time_series(real_input, gen_output))

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(real_input, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=self.gen_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(128)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(4))
        # model.summary()

        noise = Input(shape=self.gen_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()
        
        model.add(Bidirectional(LSTM(128, activation = 'relu', return_sequences=True), input_shape=self.dis_shape))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(128, activation = 'relu')))
        model.add(TimeDistributed(Dense(1, activation = 'linear')))
        #model.summary()

        img = Input(shape=self.dis_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train_input, X_train_output) = split_time_series(self.time_series_len, self.data)

        # normalize

        # Adversarial ground truths
        valid = np.ones((batch_size,1,1))
        fake = np.zeros((batch_size,1,1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of songs
            idx = np.random.randint(0, X_train_input.shape[0], batch_size)
            real_input= X_train_input[idx]
            real_output= X_train_output[idx]

            # Sample noise and generate a batch of new songs
            noise = np.random.normal(1000, 300, (batch_size,self.time_series_len,self.feature_len))
            gen_output = self.generator.predict(noise)
            real_series = merge_time_series(real_input,real_output)
            fake_series = merge_time_series(real_input,gen_output)
            print(self.scaler.inverse_transform(fake_series[0]))

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(real_series, valid)
            d_loss_fake = self.discriminator.train_on_batch(fake_series, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake songs as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save model
            if epoch % save_interval == 0:
                self.generator.save("LSTM_generator.h5")

# Model Summary
I couldn't train the model on this online notebook so I trained it locally for 1000 epochs and uploaded the h5 file.

In [ ]:
lstmgan = LSTMGAN(5, 4, load_data())
lstmgan.train(epochs=1000, batch_size=100, save_interval=100)

Loading pretrained model

Installinging Mido Library

# Generating Melody
Generating random input and letting model predict output

In [143]:

random = np.random.normal(0,1,(1,5,4))

predict = lstmgan.generator(random)

print(predict)

tf.Tensor([[ 0.08106427 -6.513324   -8.422126   -9.677593  ]], shape=(1, 4), dtype=float32)


# Back to MIDI
Save generated melody back to a .mid file

In [ ]:
midler = MidiFile()
track = MidiTrack()
midler.tracks.append(track)
track.append(Message('program_change', program=2, time=0))
for x in range(16):
    track.append(Message('note_on', note=int(predict[0][x][0]), velocity=64, time=20))
    track.append(Message('note_off', note=int(predict[0][x][0]), velocity=64, time=20))
    midler.save('new_song.mid')